## Astr-119-Final-Project-f105w

Importing numpy and sep

In [ ]:
import numpy as np
import sep

Importing matplotlib, rcParams, %matplotlib inline, and astropy.io.fits. Then using rcParams, from matplotlib, to increase the size of all figures that will follow along in the code.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

from astropy.io import fits
rcParams['figure.figsize'] = [10., 8.]

Using astropy to directly download the fits image froom the online source.

In [ ]:
from astropy.utils.data import download_file
image_file = download_file('https://archive.stsci.edu/pub/hlsp/hudf12/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits', cache=True)

Reading and opening the contents of the fits image.

In [ ]:
hdu_list = fits.open(image_file)
hdu_list.info()

Storing the data of the image in a 2-D numpy array.

In [ ]:
image_data = hdu_list[0].data

Showing the dimensions of the numpy array onto the screen.

In [ ]:
print(type(image_data))
print(image_data.shape)

Closing the fits image now that the data is stored into a variable.

In [ ]:
hdu_list.close

Defining and storing the variables for the mean and standard deviations of the image, displaying the image to the screen using matplotlib in terms of the mean and standard deviation, and saving the figures as a png file to the desktop.

In [ ]:
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig('f105wImage.png', bbox_inches="tight",dpi=300)

Displaying some of the basic statistics of the image to the screen.

In [ ]:
print('Minimum:', np.min(image_data))
print('Maximum:', np.max(image_data))
print('Mean:', np.mean(image_data))
print('StandardDeviation:', np.std(image_data))

Measuring the varying background in space in the fits image.

In [ ]:
>>> image_data = image_data.byteswap(inplace=True).newbyteorder()
bkgrd = sep.Background(image_data)

Getting an entire mean and noise value of the image to screen

In [ ]:
print('GlobalMean', bkgrd.globalback)
print('GlobalNoise', bkgrd.globalrms)

Analyzing the background as 2-D numpy array.

In [ ]:
bkgrd_image = bkgrd.back()

Displaying the background image to the screen, and saving it as a png file.

In [ ]:
plt.imshow(bkgrd_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('f105w_bkgrd.png', bbox_inches='tight', dpi=300)

Analyzing the background noise as a 2-D numpy array.

In [ ]:
bkgrd_rms = bkgrd.rms()

Displaying the noise image to the screen and saving it as a png file.

In [ ]:
plt.imshow(bkgrd_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('f105w_noise.png', bbox='inches', dpi=300)

Subtracting the background so the objects can be detected clearly.

In [ ]:
data_sub = image_data - bkgrd

Showing the subtraction data to the screen.

In [ ]:
print(data_sub)

Using numpy and sep to detect the number of objects, setting the threshold to 1.5, and defining the error.

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkgrd.globalrms)

Showing how many objects have been detected. 8640 objects.

In [ ]:
len(objects)

Importing Ellipse from matplotlib.

In [ ]:
from matplotlib.patches import Ellipse

Plotting and showing the background-subtracted image, setting the center and color for each ellipse around the objects, and the saving the image results as a png file.

In [ ]:
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
               width=6*objects['a'][i],
               height=6*objects['b'][i],
               angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('f105w_objects.png', bbox_inches='tight',dpi=300)

Displaying the fields for the 'objects' command.

In [ ]:
objects.dtype.names

Performing circular aperture photometry while using a 3 pixel radius at objects locations, and storing them in 1-D numpy arrays per object.

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                    3.0, err=bkgrd.globalrms, gain=1.0)

Showing the results of the first ten objects.

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i,flux[i], fluxerr[i]))

Now to histogram the fluxes, we must change the array of the data to a 1-D array to interpret the data correctly.

In [ ]:
print(type(flux[i].flatten()))
print(type(fluxerr.flatten()))
print(type(flag.flatten()))
print(type(image_data.flatten()))
print(type(data_sub.flatten()))
print(type(objects.flatten()))

In [ ]:
NBINS = 3
histo = plt.hist(flux, NBINS)
histo = plt.hist(fluxerr, NBINS)

Converting to AB Magnitude

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
plt.imshow(data_sub, cmap='gray', norm=LogNorm())

clrbar = plt.colorbar(ticks=[1.e1,1.e2,4.e2])
clrbar.ax.set_yticklabels(['10','100','400'])